In [7]:
import pandas as pd

In [8]:
test_flight_profile = pd.read_csv("flight-profiles/test-flight-profile.csv")
test_flight_profile

,Phase,Duration (min),Altitude (ft),Speed (kts),Thrust (%)
0,taxi,5.0,0,0,7
1,takeoff,0.5,1500,200,100
2,climb,15.0,35000,280,85
3,cruise,60.0,35000,470,60
4,descent,20.0,0,250,30


In [9]:
engine_data = pd.read_csv("data/engine-data.csv")
engine_data

,Engine Family,Engine Model,Combustor,Fuel,Bypass Ratio,Pressure Ratio,Rated Thrust (kN),Phase,Power Setting (%),Fuel Flow (kg/s),Emission Index HC (g / kg fuel),Emission Index CO (g / kg fuel),Emission Index NOx (g / kg fuel)
0,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,TAKE-OFF,100,1.14,0.1,1.4,18.4
1,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,CLIMB OUT,85,0.95,0.2,3.6,13.6
2,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,APPROACH,30,0.34,5.3,29.1,6.5
3,CFM56,5B4,DAC-II,Jet A,5.9,27.7,120.1,IDLE,7,0.12,3.6,40.1,3.9


In [13]:
profile_duration_min = test_flight_profile['Duration (min)']
profile_duration_seconds = profile_duration_min * 60
print(profile_duration_seconds)

0     300.0
1      30.0
2     900.0
3    3600.0
4    1200.0
Name: Duration (min), dtype: float64


In [14]:
import pandas as pd 
import json
import os

# ---Reading Data Files---
test_flight_profile = pd.read_csv("flight-profiles/test-flight-profile.csv")
with open ("data/engines-data.json") as f:
    data = json.load(f)

# ---Extract the phase-wise emission for the CFM56-5B4/2P engine from the JSON datafile---
phases = data["CFM56-5B4/2P"]["Phases"]

# ---Importing Variables from Data---
phase_data = []
for phase_name, details in phases.items():
    ei  = details["Emission Indices (g/kg fuel)"]
    phase_data.append({
        "Phase": phase_name,
        "Fuel Flow (kg/s)": details["Fuel Flow (kg/s)"],
        "HC (g/kg)": ei["HC"],
        "CO (g/kg)": ei["CO"],
        "NOx (g/kg)": ei["NOx"]
    })

phases_df = pd.DataFrame(phase_data)     # Dataframe containing data for all phases

fuel_flow_rates = phases_df["Fuel Flow (kg/s)"] # Extract the fuel flow rates for All Phases (in kg per second)
ei_HC = phases_df["HC (g/kg)"]                  # Extract the Emission Index for Hydrocarbons for All Phases (g/kg of fuel)
ei_CO = phases_df["CO (g/kg)"]                  # Extract the Emission Index for Carbon Monoxide for All Phases (g/kg of fuel)
ei_NOx = phases_df["NOx (g/kg)"]                # Extract the Emission Index for Nitrogen Oxides for All Phases (g/kg of fuel)

phases = test_flight_profile["Phase"] # Extract Phase data from CSV datafile
profile_duration_min = test_flight_profile['Duration (min)'] # Duration for All Phases (minutes)


# ---Calculations--- 
profile_duration_seconds = profile_duration_min * 60  # Duration for All Phases (seconds)

fuel_burned = fuel_flow_rates * profile_duration_seconds # Fuel burned for All Phases (kg)

hc_emissions = ei_HC * fuel_burned      # HC emissions in Fuel Burned
co_emissions = ei_CO * fuel_burned      # CO emissions in Fuel Burned
nox_emissions = ei_NOx * fuel_burned    # NOx emissions in Fuel Burned

total_duration = profile_duration_seconds.sum()     # Total Duration of Flight
total_fuel_burned = fuel_burned.sum()               # Total Fuel Burned
total_co_emissions = co_emissions.sum()             # Total Carbon Monoxide Emissions
total_hc_emissions = hc_emissions.sum()             # Total Hydrocarbon Emissions
total_nox_emissions = nox_emissions.sum()           # Total Nitrogen Oxides Emissions

# ---Create a DataFrame for Total Emissions Summary---
total_data = pd.DataFrame([{
    "Phase":"Total",
    "Duration (s)":total_duration,
    "Fuel Burned (kg)":total_fuel_burned,
    "HC emissions (g)":total_hc_emissions,
    "NOx emissions (g)":total_nox_emissions,
    "CO emissions (g)":total_co_emissions,
}])

# ---Create Phase-wise Emissions Summary DataFrame---
phases_summary_df = pd.DataFrame({
    "Phase":phases,
    "Duration (s)":profile_duration_seconds,
    "Fuel Flow (kg/s)":fuel_flow_rates,
    "Fuel Burned (kg)":fuel_burned,
    "HC emissions (g)":hc_emissions,
    "NOx emissions (g)":nox_emissions,
    "CO emissions (g)":co_emissions,
})
phases_summary_df = pd.concat([phases_summary_df, total_data], ignore_index=True)


# ---Saving Outputs---
output_dir = "output/emissions"
output_file = os.path.join(output_dir, "summary.csv")

phases_summary_df.to_csv(output_file, index=False)
os.makedirs(output_dir, exist_ok=True)


In [15]:
phases_summary_df

,Phase,Duration (s),Fuel Flow (kg/s),Fuel Burned (kg),HC emissions (g),NOx emissions (g),CO emissions (g)
0,taxi,300.0,1.14,342.0,34.2,6292.8,478.8
1,takeoff,30.0,0.95,28.5,5.7,387.6,102.6
2,climb,900.0,0.34,306.0,1621.8,1989.0,8904.6
3,cruise,3600.0,0.12,432.0,1555.2,1684.8,17323.2
4,descent,1200.0,NaN,NaN,NaN,NaN,NaN
5,Total,6030.0,NaN,1108.5,3216.9,10354.2,26809.2


In [4]:
import plotly.graph_objects as go
import pandas as pd

phases_df = pd.read_csv("output/emissions/summary.csv")

phases = phases_df["Phase"]
co_emm = phases_df["CO emissions (g)"]
nox_emm = phases_df["NOx emissions (g)"]
hc_emm = phases_df["HC emissions (g)"]

fig = go.Figure([
    go.Bar(name='HC emissions', x=phases, y=hc_emm),
    go.Bar(name='NOx emissions', x=phases, y=nox_emm),
    go.Bar(name='CO emissions', x=phases, y=co_emm)
])

fig.update_layout(barmode='group', yaxis_type='log')
fig.show()